This notebook's goal is to present a concept notebook of specviz, using workflows from the [IRAF splot tool](https://iraf.net/irafhelp.php?val=splot&help=Help+Page).  The feature/workflow list for splot came from a [trello board](https://trello.com/b/glMEUlE3/iraf-splot-feature-list), but the features are copied here to frame this notebook. Note also there is discussion of some of this notebook's features in the corresponding Github Pull Request: https://github.com/spacetelescope/jdaviz/pull/133

This notebook is organized around broad themes of planned `specviz` features, and how they address specific SPLOT functionality.  All SPLOT functionality is listed here, although at the end several features are listed as "not to be implemented".

Also note that most features in this notebook are divided into sections: 
1. A specification of how this can/should be accomplished "in the notebook".  It is important to recognize that "in the notebook" assumes there is an active specviz session somewhere in the notebook that is used as the "hub" for operations, even when the operations are all Python functions that do not directly require interactivity. 
2. The desktop interface - this is more focused on "quick look" type behavior.
3. The MAST interface - this is only highlighted where distinct points need to be made from 1 or 2 (which otherwise overlap a lot).

# Loading data/creating specviz:

Spectra to be loaded in specviz should be compatible with `Spectrum1D`, which should be viewed as the prime "input" to `specviz`.  

1.  This means that in the notebook, `Spectrum1D` loaders can be used directly:

In [ ]:
import specutils

spec_url = 'https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12'
spec = specutils.Spectrum1D.read(spec_url)

This spectrum should then be natively read by the specviz helper object (for more on the "helper" concept, see [jdaviz#104](https://github.com/spacetelescope/jdaviz/issues/104#issuecomment-624740003)) using a relevant method:

In [ ]:
import jdaviz

specviz = jdaviz.Specviz()
specviz.load_data(spec)

specviz  # <- could also be specviz.show(), but in the end it's just a wrapper in `Specviz._repr_html_`

    Note that this should automatically select the `.flux` part of the spectrum in the viewer at the time `load_data` is called, *not* require the user to do something to load that spectrum. The resulting spectrum should be a plot of the `flux` in "bin" form, since for JWST the "flux in bins" interpretation of spectra is preferred.  (There should eventually be an option to show it as a connected line, but that's not mandatory initially.)
    
    To show what *should* pop up in the above cell, below I create a glue viewer that resembles what the expected specviz main window should show. The actual application will have additional features like the plugin tray and data loading via the UI, but this shows the primary science-focused interaction space (the spectrum itself):

In [ ]:
# THIS CELL IS FOR ILLUSTRATION OF WHAT THE UI LOOKS LIKE FOR EXPLAINING THE CONCEPTS, NOT INTENDED AS AN ACTUAL NOTEBOOK API FOR USERS.  IT PROBABLY NEEDS TO BE EXECUTED TO ACTUALLY SHOW ANYTHING THOUGH
import glue_jupyter
print('FYI: glupyter version:', glue_jupyter.__version__)
from glue_jupyter import jglue
from astropy.io import fits

app = jglue()
data = app.add_data(spectrum=spec)
viewer = app.profile1d(data='spectrum')
viewer

    There also needs to be some way to load from a file path. This should use the exact same mechanism but use a file path or URL:

In [ ]:
specviz.load_data(spec_url)  #<- auto-downloads from the internet

# download to a local file and load it by name
!wget --content-disposition $spec_url
specviz.load_data('spec-1323-52797-0012.fits') 

    Internally this should do the same as loading the `Spectrum1D` - i.e. first convert it to `Spectrum1D` using `read` and then load it into glue/jdaviz. That way the user can know exactly what is happening when they do it with a GUI button or at the command line.

2. In the UI, the above should be possible using the "open" or "import data" button - the intention would be for it to all behave just as described above, but via a file browser (or command-line file path) instead of via a notebook method call. Note that this means for the *first* spectrum imported (either from the command-line or the "import" button) will automatically appear a la `load_data`, but see below for the behavior for additional spectra.

### Multiple spectra in one `specviz` 

Because glupyter/glue support multiple data being loaded at the same time, it should also be possible to load data even if it's not all getting displayed initially.

1.  In the notebook this is just a different method that otherwise behaves the same as `load_data`, just doesn't replace what's being shown - E.g.,

In [ ]:
specviz.add_data('some_other_spectrum.fits') #<- this would do exactly the same as `load_data` but just not show it
# OR 
specviz.add_data('some_other_spectrum.fits', show=True)  #<- this would automatically add the flux as a new layer in the UI

The inputs to `add_data` could also be a list/iterable, which then just does the above operation over every element of the list.

2. In the UI, if the user does "import data" a second time (meaning "any time there is already some other data loaded"), the equivalent of `add_data` happens, but this should pop up a notification for the user that says something like "load successful. mark checkbox to see in viewer".

## Uncertainties/mask

If the `Spectrum1D` object loaded contains an uncertainty and/or mask attribute, these should be loaded into glue when `load_data` is called.  However, by default only the spectrum itself should be *shown* at that time. The user can show it later using standard `jdaviz` interactions if desired. We *may* wish some syntactic convenience for the notebook as well - e.g.:

In [ ]:
specviz.show_uncertainty()
specviz.show_mask()

There are different possible ways to show these components of the spectrum.  As a starting point, uncertainties may be best shown as a partially-transparent 1-$\sigma$ shaded band surrounding the spectrum.  An alternative (which might be preferred, or used as an option based on user testing) is to use (stem-less?) error bars, but the shaded band should be implemented initially.

The starting point for showing a spectrum's mask is to have masked data points not be part of the spectrum itself (i.e. the spectrum "bins" should skip over them) but have a colored "X" for where the masked point would have been if it were not masked.  An alternative (to be implemented or not following user testing) is to use a shaded red vertical band.

(For both the color should be the same color as the spectrum itself.)

## Accessing the spectrum

(only relevant for notebook/developer usage) - When a single spectrum is loaded using `load_data`, it should be easy to get that spectrum back again.  E.g.,

In [ ]:
spec = specviz.visible_spectrum  #<- a Spectrum1D object
spec.flux, spec.uncertainty, ...

However, this is ambiguous if multiple spectra are loaded (see the `add_data` discussion above). Addressing this ambiguity would be possible using the glue-level names associated with these data:

In [ ]:
spec = specviz.app.get_viewer_data('spectrum-with-some-name')  #<- this is a single spectrum because it was requested *by name* following the glue convention

To make this easier for users who don't know/care about the glue internals, the helper can have an accessor to just get all the spectrum that currently appear:

In [ ]:
specs = specviz.get_viewer_data()
spec = specs[0]  #<- this is a Spectrum1D

The `visible_spectrum` above would then be basically a shortcut for `specviz.get_viewer_data()[0]`, although it *might* also raise an exception if more than one spectrum is showing as a way for the user to be cued into the idea that they need to account for the possibility of multiple spectrum being shown.

So then in the below, `specviz.visible_spectrum` is used for simplicity, but it should all work the same if `specviz.get_viewer_data()[0]` is substituted.

 ## SPLOT commands addressed by data loading operations 

### Options: images

*Description*:

The actual spectrum file that gets loaded when splot starts.

*Note* "images" here is a term SPLOT uses which seems to imply 2d spectra or cubes, but in this context means roughly "a fits file with spectra in it".

### g -- get another spectrum

*Description*:

The current spectrum is replaced by the
        new  spectrum.   The  aperture/line  and  band  are  queried  is 
        necessary.


### Query: next_image

*Description*:



# Zoom/Pan

The UI should support standard plot interaction options, including:

* Zooming *vertically* (without zooming horizontally)
* Zooming *horizontally* (without zooming vertically)
* A "reset to initial view" option - i.e., show the whole spectrum.
* Box-select zooming (implicitly also panning)
* Standard "hand"/touch-style panning
* Center view on click/keystroke (optional/lower priority)

### Explicitly specifying limits

There should also be a way to manually set the plot limits:

1. In the notebook - e.g.:

In [ ]:
specviz.y_limits = (0, 1000) # <- uses whatever the current units are

# note this is already available as part of the glue viewer, so this should be syntactic sugar for something like:
specviz.spectrum_viewer.state.y_min = 0
specviz.spectrum_viewer.state.y_max = 1000

    The notebook interface should also understand units.  That is, this should set the view to 5000 - 7000 angstroms regardless of what units the spectrum itself is in:

In [ ]:
from astropy import units as u

specviz.x_limits = (5000*u.angstrom, 7000*u.angstrom)

# if possible this should *also* defer to glue for unit-handling, but if that is non-trivial it could be part of the helper class

    The above should *also* support `SpectralRegion`s as a convenience.  E.g., the above should be the same as:

In [ ]:
region = specutils.SpectralRegion(5000*u.angstrom, 7000*u.angstrom)
specviz.x_limits = region

    Closely related, it should be possible to "flip" the x-axis.  With manual specification this should be as simple as:

In [ ]:
specviz.x_limits = (7000*u.angstrom, 5000*u.angstrom)

    But it might also be useful to have a convenience:

In [ ]:
specviz.flip_x()

Note that an option to flip the x-axis is [ALREADY IMPLEMENTED] in glupyter, but the above may be convenient syntactic sugar

2. In the UI, in general the above should use glue/bqplot's native controls for this, and this functionality should be added where it does not exist (although unit-specification support might be complex enough to treat as a stretch goal for the UI).

#### Autoscaling:

There should also be autoscale options which means "go from the beginning to end".  

1. In the notebook, this might be as simple as:

In [ ]:
specviz.reset_view()

#which should be equivalent to:

specviz.x_limits = 'auto'
specviz.y_limits = 'auto'

# or similar

    Additionally, to address the SPLOT `b` command use case, it should be possible to do:

In [ ]:
specviz.y_limits = (0, 'auto')

    Which would mean "go from 0 to wherever the highest y value is in the spectrum"

    Additionally, autoscaling might be better done as a method, because we may want more complicated ways of doing autoscaling:

In [ ]:
specviz.autoscale_y()  #<- same as above
specivz.autoscale_y(sigma_clipping=True)  #<- y-scale using sigma-clipping to get the range to view - useful when a spectrum has a few artifact outliers  

2. These notebook commands should eventually be exposed in the UI, probably via a plugin, although those using the glue machinery will already be available via glue itself.

## SPLOT commands addressed by pan/zoom:

### a -- Expand and autoscale to the data range between two cursor positions

*Description*:



### b -- Set the plot base level to zero rather than autoscaling

*Description*:



### c -- clear all windowing and redraw the current full spectrum.

*Description*:

Clear  all windowing and redraw the full current spectrum.  This
        redraws the spectrum and cancels any effects of  the  'a',  'z',
        and  'w'  keys.  The 'r' key is used to redraw the spectrum with
        the current windowing.
    


### r -- Redraw  the  spectrum with the current windowing.

*Description*:

Redraw  the  spectrum with the current windowing.  To redraw the
        full spectrum and cancel any windowing use the 'c' key.

### w -- window the graph

*Description*:

Window the graph.  For further help type '?'  to  the  "window:"
        prompt  or  see  help under gtools.  To cancel the windowing use
        'a'.

### z -- Zoom the graph by a factor of 2 in x.

*Description*:



### , -- Shift the graph window to the left.

*Description*:



### . -- Shift the graph window to the right.

*Description*:



### Options: wcreset

*Description*:

Option 
        "wreset"  resets  the  graph  limits  to  those specified by the
        xmin, xmax, ymin, ymax parameters whenever  a  new  spectrum  is
        plotted.  

### Options: flip

*Description*:

The  "flip" option selects that initially the spectra
        be plotted with decreasing  wavelengths. 

### Options: auto

*Description*:

Option  "auto"
        automatically  replots  the  graph  whenever  changes  are made.
        Otherwise the graph is replotted with  keystrokes  'c'  or  'r'.

### Options: zero

*Description*:

Option  "zero"  makes  the initial minimum y of the graphs occur
        at zero.  Otherwise the limits are set  automatically  from  the
        range  of  the  data  or  the  ymin  parameter. 

### Initial limits of graph (xmim,xmax,ymin,ymax)

*Description*:

 The  default  limits  for  the initial graph.  If INDEF then the
        limit is determined from the range of  the  data  (autoscaling).
        These  values  can  be changed interactively with 'w' window key
        options or the cursor commands ":/xwindow" and ":/ywindow"  (see
        gtools).


# Measurements over interactively-specified spectral regions

Many science use cases depend on measurements of parts of a spectrum like spectral lines, which are most easily specified interactively in "quick look" situations.  Specviz should support this use case by:

1. For notebooks, providing an easy way to extract SpectralRegion objects from selections made in the specviz UI. This would most straightforwardly be accomplished by making a subset selection, and then providing a way to convert that into a SpectralRegion.  E.g., if the user uses the horizontal-selection to create a subset,  they can do:

In [ ]:
reg = specviz.get_selected_spectral_region()  #<- uses the "most recent" selection
reg2 = specviz.get_selected_spectral_region('Subset 2')  #<- uses glue's name for the subset

all_regs = specviz.get_selected_spectral_region(all=True)  #<- this would be *all* the subsets combined together into a single SpectralRegion object, which is [ALREADY IMPLEMENTED] by SpectralRegion (but not specviz).  
# See the "fractional_pixel_extraction" jdaviz notebook for an example of the above

    Note that it is critical for some use cases that the above should yield spectral regions that are correct *even to the sub-pixel*.  That is, if the user does a region selection that ends between two pixels, that should be reflected in the SpectralRegion - this means using subset *states* in glue instead of the subset itself.

    These selections can now be used with standard specutils analysis functions in a notebook setting (these are [ALREADY IMPLEMENTED]):

In [ ]:
specutils.analysis.line_flux(specviz.visible_spectrum, specviz.get_selected_spectral_region())

specutils.analysis.equivalent_width(specviz.visible_spectrum, regions=specviz.get_selected_spectral_region())

specutils.analysis.gaussian_fwhm(specviz.visible_spectrum, specviz.get_selected_spectral_region())

specutils.analysis.snr(specviz.visible_spectrum, specviz.get_selected_spectral_region())

    In addition to region selection, there is value in having a way to extract a specific point in the displayed spectrum into the notebook. (Some examples are shown below) - while there are many potential solutions to this, the simplest is a stateful/blocking "get location" method.  E.g.,

In [ ]:
spectral_coordinate, flux_coordinate = specviz.get_marked_location()

    This method would use a marked location that the user would mark using a special singleton marking tool. (and raise an exception if no mark is present).  This is the default thing to try first. 
    
    An alternative (to be tried if the above does not meet user testing or is harder to implement) is to have this not be stateful but rather report on the last clicked location in the specviz plot.  Another alternative is to have a special single-pixel selection mode. Or it could be a "blocking" tool that locks up the kernel until the user clicks on a point (a la SPLOT).

2. In the Desktop interface, relevant analysis functions like the above should be exposed as information plugins - when run this plugin would simply show the result of these functions as run in the latest `get_spectral_region` subset exactly as shown above but auto-update when the user creates a new/updates the current subset.

    One additional related piece of functionality that should be added to the above plugins: provide a way to replicate the `e` key feature below.  That is, provide a quick way for the user to not just specify the spectral region, but also mark two points to do automatic continuum-specification.  This defines a simple linear continuum, which the plugin could subtract from and divide from the spectrum before doing the calculation desired.  How exactly the user specifies the two points is TBD, but perhaps could be a glue box-selection, or a special two-click stateful operation particular to the plugin.  
    
    Note that in *addition* to this there should eventually be a way to do more thorough model + continuum fitting options as outlined further below, but the emphasis here is on a quick-and-easy version like the `e` key of splot that doesn't require full understanding of continuum fitting. 
    
3. Note that in the MAST interface, a lot of the same functionality is needed, although probably more simplified versions - to see a bit more detail see https://innerspace.stsci.edu/display/ASB/Rank+list+of+features, Functional Features - Visualization, Spectral Manipulation, and Spectral Analysis.

## SPLOT commands related to region-based measurements addressed above

## e -- Measure equivalent width

*Description*:

Measure equivalent width by marking two continuum points  around
        the  line  to  be  measured.  The linear continuum is subtracted
        and the flux is determined by simply  summing  the  pixels  with
        partial  pixels  at  the  ends.   Returned  values  are the line
        center, continuum at the region center, flux above or below  the
        continuum, and the equivalent width.

### h -- Measure  equivalent  widths assuming a gaussian profile with the         width measured at a specified point.

*Description*:

Measure  equivalent  widths assuming a gaussian profile with the
        width measured at a specified point.  Note that this  is  not  a
        gaussian  fit (see 'k' to fit a gaussian)!  The gaussian profile
        determined here may be subtracted with the '-'  key.   A  second
        cursor key is requested with one of the following values:

        a   Mark  the  continuum  level  at  the line center and use the
            LEFT half width at the half flux point.
        
        b   Mark the continuum level at the  line  center  and  use  the
            RIGHT half width at the half flux point.
        
        c   Mark  the  continuum  level  at  the line center and use the
            FULL width at the half flux point.
        
        l   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use  the LEFT width at that flux
            point.
        
        r   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use the RIGHT width at that flux
            point.
        
        k   Mark  a  flux  level  at  the  line  center  relative  to  a 
            normalized  continuum  and  use  the FULL width at that flux
            point.

### m -- Compute the mean, RMS, and signal-to-noise over a region  marked         with two x cursor positions.

*Description*:



## SPACE -- prints  the  cursor  position  and value of the nearest pixel.

*Description*:



This could be exposed in the UI but in that case `jdaviz` is more consistent with a continuously-updated value instead of a "only when a keystroke fires" implementation.  

TBD question: should it be a plugin, or (optional) part of the main UI/viewer?

# Fitting-related functionality 

The below discussion includes both fitting in the sense of fitting individual spectral lines *and* continuum fitting.  Both use the same machinery, but with different appropriate models.

1. For a notebook, most of the fitting-related functionality is best implemented via specutils and Astropy's model-fitting machinery applied to the specviz spectrum. All but the specviz-accessing parts are [ALREADY IMPLEMENTED]. E.g., to fit a order-12 legendre polynomial to a continuum while excluding the H$\alpha$ region, and then using it to subtract the continuum from the existing specviz spectrum, one could do:

In [ ]:
continuum = specutils.fitting.fit_continuum(specviz.visible_spectrum, 
                                            model=astropy.modeling.models.Legendre1D(12),
                                            exclude_regions=[specutils.SpectralRegion.from_center(6563*u.angstrom, 30*u.angstrom)])

continuum_subtracted_spectrum = specviz.visible_spectrum - continuum(specviz.visible_spectrum.spectral_axis)

specviz.load_data(continuum_subtracted_spectrum)

    An almost-identical interface can be used to fit spectral lines.  For example, to fit a voigt profile to the most-recently selected subset:

In [ ]:
specutils.fitting.fit_lines(specviz.visible_spectrum,
                            astropy.modeling.models.Voigt1D(),
                            window=specviz.get_selected_region())

2. for the desktop interface, A fitting UI is necessary for the above steps, which is currently in-progress for cubeviz.  The same model-fitting UI should be adopted for specviz.  Missing pieces that are needed to replicate SPLOT functionality include:

    * A way to generate a new spectrum in specviz that comes from subtracting a fitted continuum from the current spectrum (overlaps with the arithmetic discussion below)
    * A way to generate a new spectrum in specviz that comes from dividing by a fitted continuum from the current spectrum (overlaps with the arithmetic discussion below)
    * Functionality to allow sigma-clipping as part of the fitting process (bonus/stretch goal to actually *show* the clipped points)
    * Functionality to allow manual rejection of specific points to be excluded in the fit.
    
    Note that the first two are available trivially from the notebook interface.  The latter two area also possible from a notebook but not necessarily easy.

## SPLOT commands around fitting addressed above

### t -- fit a function to the spectrum.

*Description*:


Fit a function to the spectrum using the ICFIT mode.   Typically interactive  rejection is used to exclude spectra lines from the fit in order to fit a  smooth  continuum.   A  second  keystroke selects what to do with the fit.
        
        /   Normalize  by  the  fit.   When  fitting  the continuum this
            continuum normalizes the spectrum.
        
        -  Subtract the fit.  When fitting the continuum this  continuum
           subtracts the spectrum.
        
        f   Replace the spectrum by the fit.
        
        c   Clean  the  spectrum by replacing any rejected points by the
            fit.
  
        n   Do the fitting but leave the spectrum unchanged  (a  NOP  on
            the  spectrum).   This  is  useful to play with the spectrum
            using the capabilities of ICFIT.
        
        q   Quit  and  don't  do  any  fitting.   The  spectrum  is  not 
            modified.


### d -- Mark two  continuum  points  and  fit  (deblend)  multiple  line         profiles.

*Description*:

Detailed description in the splot help file...


### k -- Mark two continuum points and fit a single  line  profile.

*Description*:

Mark two continuum points and fit a single  line  profile.   The
        second  key  selects  the type of profile: g for gaussian, l for
        lorentzian, and v for voigt.  Any other second key  defaults  to
        gaussian.   The center, continuum at the center, core intensity,
        integrated flux, equivalent width, and  FWHMs  are  printed  and
        saved  in  the  log file.  See 'd' for fitting multiple profile
        and '-' to subtract the fit.

### - -- Subtract the fit

*Description*:

Subtract the fits generated by the 'd'  (deblend),  'k'  (single
        profile  fit),  and  'h'  (gaussian  of  specified  width).  The
        region to be subtracted is marked with two cursor positions.

### Fitting: function

*Description*:

Function   to   be  fit  to  the  spectra.   The  functions  are 
        "legendre"   (legendre   polynomial),   "chebyshev"   (chebyshev  
        polynomial),  "spline1"  (linear  spline),  and "spline3" (cubic
        spline).  The functions may be abbreviated.


### Fitting: order

*Description*:

**order**
        The order of the polynomials or the number of spline pieces.

### Fitting: low_reject, high_reject

*Description*:

Rejection limits below  and  above  the  fit  in  units  of  the
        residual  sigma.   Unequal  limits  are  used to reject spectral
        lines on one side of the continuum during continuum fitting.


### Fitting: niterate

*Description*:

Number of rejection iterations.

### Fitting: grow

*Description*:

When a pixel is rejected, pixels within  this  distance  of  the
        rejected pixel are also rejected.


### Fitting: markrej

*Description*:

Mark  rejected  points?   If  there  are many rejected points it
        might be desired to not mark rejected points.

# Arithmetic-related operations 

This section deals with arithmetic between multiple spectra.

1. In the notebook, this is trivial, as `Spectrum1D` objects support arithmetic.  E.g., subtracting a smoothed version of a spectrum from the spectrum and showing it can be done as:

In [ ]:
specviz.load_data(specviz.visible_spectrum - box_smooth(specviz.visible_spectrum, 10))

2. In the UI, this is potentially much more complicated, as a plugin to support arithmetic would be necessary. Such a plugin is possible, but not necessarily desirable, as part experience suggests the feature requests for this quickly spiral, suggesting simply saying "use the notebook" is a better answer.

    Note, though: two piece of arithmetic are critical: a way to subtract a fitted continuum and a way to divide by it.  These could be implemented as part of the *fitting* plugin, however, to avoid having to write a separate arithmetic plugin for this purpose. So the baseline specviz should include options for subtracting/dividing by continuum in the fitting UI, and a more general arithmetic plugin (which *must* use the same underlying `specutils` machinery to not confuse users) is an option to consider later.

## SPLOT commands related to arithmetic addressed above

### Query: Spec2

*Description*:



### Query: constant

*Description*:



### f -- Arithmetic function mode

*Description*:


Enter  arithmetic  function  mode.  This  mode allows arithmetic
        functions to be applied to the spectrum. The  pixel  values  are
        modified  according  to the function request and may be saved as
        a new spectrum with the 'i' command.  Operations with  a  second
        spectrum  are  done  in wavelength space and the second spectrum
        is automatically resampled if necessary.   If  one  spectrum  is
        longer  than  the  other,  only the smaller number of pixels are
        affected.  To exit this mode type 'q'.
        
The keystrokes listed in the checklist are available  in  the  function  mode.
        Binary  operations  with a constant or a second spectrum produce
        a query for the constant value or spectrum name.
        

# Unit-related operations

Because `Spectrum1D` objects carry their units and spectral axes, many of the unit operations in SPLOT are implicit by virtue of having units associated with a spectrum. Additionally `glue` natively understands the concept of world coordinates (i.e., the spectral coordinate) and corresponding pixel coordinates.  Hence SPLOT-related functionality of setting wavelengths units or scales is most straightforwardly addressed in a notebook setting:

1. For example, if a user starts with a spectrum that has only pixels but wants to specify a linear wavelength axis, they can simply create one in a new Spectrum1D:

In [ ]:
new_spec = specutils.Spectrum1D(flux=specviz.visible_spectrum.flux,
                     spectral_axis=np.linspace(4000*u.angstrom,7000*u.angstrom, specviz.visible_spectrum.flux.size))
specviz.load_data(new_spec)

And an even simpler operation to convert the flux units:

In [ ]:
specviz.load_data(specviz.visible_spectrum.new_flux_unit(u.Jy))

To change the *display* of the spectrum using specviz is less trivial, but should be possible - while a glue-style approach is feasible:

In [ ]:
specviz.app.get_viewer().state.x_att = specviz.app.data_collection.data[0].components[0]

A simple method that reproduces the above might be better:

In [ ]:
specviz.use_pixel_axis()
specviz.use_spectral_axis()

2. Units should be *displayed* where available. Unit conversion in both the spectral_axis and the flux axis (including f_nu to f_lambda) should also be possible - ideally this is either the user types a unit in (using astropy units parsing), or a dropdown of common spectral units, or better yet: both (i.e. a dropdown with common units that can also be free-hand editable text - although this may or may not be an already existing vue/ipy widget?).  Note that it is a mandatory requirement that f_nu/f_lambda conversions *not* change the x-axis units unless specifically required, and therefore that this plugin support the y-axis unit conversions and x-axis conversions independently.

## SPLOT commands related to unit operations addressed above

### $ -- Switch    between   physical   pixel   coordinates   and   world           (dispersion) coordinates.

*Description*:



### p -- define a linear wavelength scale

*Description*:

Define  a  linear  wavelength  scale.  The user is queried for a
        starting  wavelength  and  an  ending  wavelength.   If   either 
        (though  not  both)  are  specified  as  INDEF  a  dispersion is
        queried for and used  to  compute  an  endpoint.   A  wavelength
        scale  set this way will be used for other spectra which are not
        dispersion corrected.
    

### :units -- Change the coordinate units in the plot.

*Description*:



### Options: units

*Description*:


        Dispersion coordinate units for the plot.  If the  spectra  have
        known  units,  currently  this is generally Angstroms, the units
        may be converted to other units for  plotting  as  specified  by
        this  task  parameter.  If this parameter is the null string and
        the  world  coordinate  system  attribute   "units_display"   is 
        defined  then  that  will be used.  If both this task parameters
        and "units_display" are not given then the  spectrum  dispersion
        units   will   be   used.    The   units  may  also  be  changed 
        interactively.  See the units section of the package help for  a
        further description and available units.


### l -- convert to f-lambda

*Description*:

Convert to flux per unit wavelength (f-lambda). The spectrum  is
        assumed  to  be  flux  calibrated  in  flux  per  unit frequency
        (f-nu).  See also 'n'.

### n -- convert to fnu

*Description*:



# Redshifting, line lists, and related spectral coordinate shifts 

There should be a redshift slider (optional?) somewhere in the specviz UI.  This slider should update the redshift/radial_velocity recorded in the spectrum's `.spectral_axis.redshift`/.spectral_axis.radial_velocity` attributes.

1. In the notebook these can be accessed directly as informational items:

In [ ]:
specviz.visible_spectrum.redshift, specviz.visible_spectrum.radial_velocity

    Similarly, the functionality of the `u` and `v` keys are also available in the notebook as long as the pixel location under the cursor is available - e.g., to do the shift that corresponds to `u` followed by `z` and `6563` as the rest wavelength:

In [ ]:
obs_wl = specviz.get_marked_location()[0]  #<- blocks until the user clicks a line
specviz.visible_spectrum.redshift = obs_wl/6563-1

Similar operations on `specviz.visible_spectrum.spectral_axis` allow `v` or other more complex workflows.

2. A plugin could be written to handle workflows like `u` and `v` below and have them set the redshift appropriately.  It is not clear these are common enough to need to be in a plugin as opposed to directing users to the notebook as stated above, however.

### Line lists 

While the redshift is useful contextual information, it is most important when used to identify spectral lines.  For this purpose the user needs to be able to input line lists appropriate for their science case. The below describe how to specify such line lists, but regardless the intent is for the user to then see those lines displayed in `specviz`, redshifted to wherever they should be given the value of the `.redshift`/`.radial_velocity` attribute.

1. In the notebook line lists should be loadable straightforwardly as astropy tables with spectrally-appropriate units.  The most straightforward option is to consume any table that has 'linename' and 'rest' columns:

In [ ]:
linelist_table = astropy.table.Table()
linelist_table['linename'] = ['Halpha', 'Hbeta']
linelist_table['rest'] = [6563, 4862]*u.angstrom
linelist_table['more information'] = ...

specviz.load_linelist(linelist_table)

2. In the UI, either a plugin or a pane specifically dedicated to this (perhaps associated with the "redshift" slider?) should present a few options: a "galaxies" linelist, a "stars" linelist, and a "load file..." option.  The first two would be curated lists (ideally in `specutils` but could be in `jdaviz`), while "load file..." would prompt the user for a file, which would then be read as `astropy.table.Table.read('/path/to/file')`.  This ensures the same line list tables can be used in the notebook and the UI setting.

Regardless of the above, in the end vertical lines should appear at the locations specified in `'rest'`, and the UI should have at least a few styling options exposed (although that may be a lower-priority stretch goal in detail).

### Marking lines

Relatedly, there should be a way to quickly and easily mark a specific point in a spectrum and say "find the line near here".

1. In the notebook this should be a method that follows the some workflow as the `get_marked_location` mentioned above, but combined with the `find_lines` and `centroid` functions in `specutils`.  E.g., the user would do something like:

In [ ]:
line_center = specviz.mark_line(name='Halpha', line_finding_window=50*u.angstrom, ...other options for `find_lines` or `centroid`)

    Which would then use find_lines to get a first guess and `centroid` to identify the center.  This would then get registered in specviz as one of the lines in this spectrum, which would allow something like:

In [ ]:
z = specviz.guess_redshift_from_lines()
spec.redshift = z

    Which would find which marked lines match the linelist and use that to produce an estimate of the redshift. Note that this should all be "syntactic sugar" and clearly indicate in the docstring how the user might reproduce this workflow on their own if they want to e.g. use their own `centroid` function that works best for their specific data.

2. The above workflow should be available in a plugin for redshifting, which might *also* have options for x-corr redshifting (details of that TBD, but it is logically closely connected).

### Rest/observed conversions

Another important use case after the above are addressed is the ability to show a "rest-frame" x-axis.

1. In the notebook, `specutils.spectral_axis` is now a `SpectralCoord` object that [already understands rest/observed conversions](https://docs.astropy.org/en/latest/coordinates/spectralcoord.html#reference-frame-transformations). For operations that require the rest x-axis, this should just require:

In [ ]:
rest_axis = spec.spectral_axis.to_rest()
... do something with rest_axis ...

2. In the UI there should should be a toggle box somewhere to toggle between rest and observed.  This would use the `SpectralCoord` machinery exactly as shown above, updated as the redshift slider is moved.  A better alternative is to show *both* the rest and observed on the top and bottom, but that is not mandatory if it is technically difficult. Either way, the toggle of the UI state should also be accessible via a convenient helper along the lines of:

In [ ]:
specviz.show_rest_spectral_axis()
specviz.show_observed_spectral_axis()

(And the vertical lines for any loaded line list then should appear at their appropriate *rest* locations)

## SPLOT commands related to redshifting operations addressed above

### u -- Adjust  the user coordinate scale.

*Description*:

Adjust  the user coordinate scale.  There are three options, 'd' mark a position with the  cursor  and  doppler  shift  it  to  a specified  value,  'z'  mark  a  position  with  the  cursor and zeropoint shift it  to  a  specified  value,  or  'l'  mark  two positions   and   enter   two  values  to  define  a  linear  (in  wavelength) dispersion scale.  The  units  used  for  input  are those  currently  displayed.   A  wavelength  scale set this way will  be  used  for  other  spectra  which  are  not  dispersion  corrected.

### v -- Toggle  to  a velocity scale using the position of the cursor as         the velocity origin and back.

*Description*:



# Functionality for multidimensional spectra

SPLOT includes functionality (partly from broader IRAF) for doing spectral operations on multidimensional arrays like "2d" spectra or data cubes.  This is intentionally not a target for `specviz` so it is not a priority of the UI. Instead, `mosviz` and `cubeviz` target such datasets.  

That said, much of the multidimensional SPLOT workflows can be addressed from the notebook by slicing spectrum objects (this does not address 2d or cube views, but SPLOT does not either):

In [ ]:
import numpy as np
spectrum2d = np.random.randn(50, 1024) # A fake "2d spectrum" to approximate a 50-pixel slit with 1024 spectral pixels along columns

spec = specutils.Spectrum1D(spectral_axis=np.arange(1024)*u.pixel,
                            flux=spectrum2d[25]*u.adu)
specviz.load_data(spec)

Or alternatively, if a data cube or other construct can be loaded as a `SpectrumCollection`, it could simply be loaded as:

In [ ]:
speccoll = specutils.SpectrumCollection.read('...') # assume this is a 50x50xn_spec data cube

specviz.load_data(speccoll[20, 30])  # shows the 20, 30th spaxel

The above could potentially be exposed as part of a custom file-loading UI for the desktop tool, but it is not clear that this is a priority so should probably be considered a "nice to have" or "stretch goal".

## SPLOT commands addressed by multidimensional loading or mosviz

### Options: line and band: 

*Description*:

One  may  use  an         image  section to select a desired column, line, or band

**line, band**
        The  image  line/aperture  and  band  to  plot  in  two or three
        dimensional images.   For  multiaperture  spectra  the  aperture
        specified  by  the  line  parameter  is  first sought and if not
        found the specified image  line  is  selected.   For  other  two
        dimensional   images,  such  as  long  slit  spectra,  the  line 
        parameter specifies a line or column.  Note  that  if  the  line
        and  band  parameters  are specified on the command line it will
        not be possible to change them interactively.


### ( -- In multiaperture spectra go to the  spectrum  in  the  preceding         image  line.

*Description*:

In multiaperture spectra go to the  spectrum  in  the  preceding
        image  line.   If  there  is only one line go to the spectrum in
        the preceding band.

### ) -- In multiaperture spectra go to the  spectrum  in  the  following         image  line.

*Description*:

In multiaperture spectra go to the  spectrum  in  the  following
        image  line.   If  there  is only one line go to the spectrum in
        the following band.

### q -- quit and go to the next input spectrum

*Description*:

Quit and go on to next input spectrum.  After the last  spectrum
        exit.

### # -- Get  a  different  line  in   multiaperture   spectra   or   two          dimensional images.

*Description*:

Get  a  different  line  in   multiaperture   spectra   or   two 
        dimensional images.  The aperture/line/column is queried.

### % -- Get a different band in a three dimensional image.

*Description*:



### :dispaxis <val> -- Show or change dispersion axis for 2D images.

*Description*:



### :sum -- Show or change summing for 2D images.

*Description*:



# Other implemented/to-implement SPLOT functionality

## Options: histogram

*Description*:

Option "histogram"  plots  the  spectra  in  a  histogram  style
        rather  than  connecting  the  pixel  centers. 

### Specviz Solution

`specviz` should default to histogram style (although there's some disagreement about this), and it's a "nice to have" option to be able to toggle between that and line-connected style.  But either way this is non-critical.

## : log, nolog -- Enable/Disable  logging  of  measurements  to  the file specified by the parameter save_file.

*Description*:



### Options: save_file

*Description*:

save_file
        The  file  to  contain  any  results generated by the equivalent
        width or deblending functions.  Results are added to  this  file
        until  the  file is deleted.  If the filename is null (""), then
        no results are saved.


#### :# -- Add comment to logfile

*Description*:



#### :show -- Page the full output of  the  previous  deblend  and  equivalent         width measurements.

*Description*:



### Specviz Solution


`specviz` will meet the logging need in two ways:

1. For notebook-oriented workflows, the notebook itself can the "save file" - e.g., computing the equivalent width using a `SpectrumRegion` that's explicitly specified already records the result for reproducibility. [ALREADY IMPLEMENTED]
2. For desktop tool workflows (or notebook workflows in part using plugins), a log should be available to track operations. This could/should use the standard Python logging functionality, but to reproduce the specific needs of SPLOT, plugins should have a way to explicitly say "log my results". This log file should be accessible in the UI which would then have a "save" option for the user to record this if they want to. Note it is not clearly exactly what workflows require this so it's somewhat "nice-to-have", and probably also needs a start/stop option (or at least a way to clear and then save only what came after clearing).

## Labels: various forms of labeling

*Description*:

    Labels:

        :label <label> <format>
            Add a label at the cursor position.
        
        :mabove <label> <format>
            Add  a  tick mark and label above the spectrum at the cursor
            position.
        
        :mbelow <label> <format>
            Add a tick mark and label below the spectrum at  the  cursor
            position.
        
        The  label  must  be  quoted  if  it  contains  blanks.  A label
        beginning with % (i.e. %.2f) is treated as a format  for  the  x
        cursor  position.   The  optional  format is a gtext string (see
        help on "cursors").   The  labels  are  not  remembered  between
        redraws.

### Specviz Solution


`bqplot` and `glue`/`glupyter` have mechanisms for setting labels.  It might be a bonus to add convenient accessors to make setting such labels easier along the lines of the cell below (but not this is not critical functionality):

In [ ]:
specviz.set_flux_label('My Nobel Prize-winning spectrum', include_unit=False, ... other plotting options passed on to glue...)
specviz.set_spec_label('My special name for the spectral axis', include_unit=True, ... other plotting options passed on to glue...)

## i --  Write the current spectrum out to a new or existing image.

*Description*:



#### Query: new_image (for saving results of spectrum arithmetic)

*Description*:

 In response to  'i'  (write  current  spectrum)  this  parameter
        specifies  the  name  of a new image to create or existing image
        to overwrite.

Modifications  to  the spectra being analyzed may be saved using the
    'i' key in a new, the current, or other  existing  spectra.   A  new
    image  is  created  as  a new copy of the current spectrum and so if
    the  current  spectrum  is  part  of  a  multiple   spectrum   image 
    (including  a  long slit spectrum) the other spectra are copied.  If
    other spectra in the same image are then modified and saved use  the
    overwrite  option  to  replace then in the new output image.  If the
    output spectrum already exists then the overwrite flag must  be  set
    to  allow  modifying  the  data.   This  includes  the case when the
    output spectrum is the same as the input  spectrum.   The  only  odd
    case  here  is  when  the  input spectrum is one dimensional and the
    output spectrum is two  dimensional.   In  this  case  the  user  is
    queried for the line to be written.


#### Query: overwrite

*Description*:



### Specviz Solution


The above three can be addressed by:

1. In the notebook, `Spectrum1D.write` is the best way to write out a spectrum in a standard way.
2. In the desktop interface, a plugin to do `Spectrum1D.write` on a loaded spectrum is an option.  *However* it is not clear that this meets any needs if the arithmetic operations are not exposed in the UI (see the above discussion), so this should only be implemented if this is done and/or there is some other identified need. However, the outputs of *analysis* operations (as opposed to the modified spectrum itself) are important to be saveable, which should be done on a plugin-by-plugin basis.

### j -- Set  the  value  of  the  nearest pixel to the x cursor to the y         cursor position.

*Description*:



In [ ]:
specviz.visible_spectrum.flux[pixel_shown] = desired_value

In principle this is [ALREADY IMPLEMENTED] *but* it is not clear to me if the current implementation allows in-place setting - that should be checked.

2. In the UI this could be implemented as a plugin or a special selection mode, although it is not at all clear that this is that useful for any known "desktop-specific" use case if the notebook option above is supported and documented.

## s -- smooth via boxcar.

*Description*:

Smooth via a boxcar.  The user is prompted for the box size.

### Specviz Solution


1. In a notebook, this is done via `specutils` smoothing functionality, which then would be given as an updated spectrum:

In [ ]:
smoothed_spec = specutils.manipulation.box_smooth(specviz.visible_spectrum, width)
specviz.add_data(smoothed_spec, show=True)  # <- to overplot
specviz.load_data(smoothed_spec)  #<- to replace the current spectrum

2. In the UI this can be implemented via a smoothing plugin that does the above [ALREADY IMPLEMENTED]

# Specviz functionality in mosviz or cubeviz

Note that some of the functionality here is also desirable in cubeviz or mosviz since both have a "specviz view". While the plugin architecture means plugins built for specviz can also work in other `jdaviz` configurations (as long as the viewers are named appropriately), the same does not hold for the notebook interactions outlined above.  For those, however, the below idiom should be used:

In [ ]:
cubeviz = jdaviz.Cubeviz(...)
... do whatever is necessary to load the relevant cubeviz data ...

specviz = jdaviz.Specviz(app=cubeviz.app)
specviz.autoscale_y() #<- assuming the viewer in cubeviz is the same name, this should work transparently

# Won't-implement SPLOT features/options

### x -- "Etch-a-sketch"  mode.  Straight   lines   are   drawn   between          successive  positions  of the cursor.

*Description*:

"Etch-a-sketch"  mode.  Straight   lines   are   drawn   between 
        successive  positions  of the cursor. Requires 2 cursor settings
        in x.  The nearest pixels are used as the endpoints.  To draw  a
        line  between  arbitrary  y  values  first use 'j' to adjust the
        endpoints or set the "xydraw" option.
        
**NOTES** It's not clear any science use case really needs this, and if desired it is better to be done using plotting machinery directly.

### Options: xydraw

*Description*:

Option "xydraw"
        changes the 'x' draw key to use both x and y cursor  values  for
        drawing  rather  than  the  nearest pixel value for the y value.
        
**NOTES** (see `x` above)

# Not-relevant-for-jdaviz SPLOT features/options 

### I -- Force a fatal error interrupt to leave the graph.

*Description*:

Force a fatal error interrupt to leave the graph.  This  is  used
        because  the  normal  interrupt  character is ignored in graphics
        mode.
        
**NOTES** There is no stateful "graph" in either the desktop tool or a notebook, so there is no need to have a special way to trigger an interrupt.

### Options: nosysid

*Description*:

Option "nosysid"
        excludes  the  system  banner  from  the  graph  title.
        
**NOTES**: No relevant "system banner" in jdaviz

### o -- set overplot flag / Options: overplot

*Description*:

Set  overplot  flag.   The  next  plot will overplot the current
        plot.  Normally this key is immediately followed by one of  'g',
        '#',  '%',  '(',  or  ')'.   The  ":overplot"  colon command and
        overplot parameter option may be used to set overplotting to  be
        permanently on.
        
The  "overplot"  options 
        overplots all graphs and a screen erase  only  occurs  with  the
        redraw key.

**NOTES**: Natively supported by glue, but follows a different idiom ("layers"), so this way of thinking does not apply.  Instead multiple plotting layers can be turned on and off in the glupyter interface (UI or programmatically).

### Options: nerrsample (number of samples for error calculation)

*Description*:

nerrsample = 0
        Number of samples for the error computation.  A value less  than
        10  turns  off  the  error  computation.   A value of ~10 does a
        rough error analysis, a value of ~50  does  a  reasonable  error
        analysis,  and a value >100 does a detailed error analysis.  The
        larger this value the longer the analysis takes.
        
        
**NOTES** Not relevant for specviz because uncertainty handling is part of the fitting/etc process not a global property of the viz tool

### Options: sigma0 (used for calculating uncertainties)

*Description*:

** sigma0 = INDEF, invgain = INDEF **
The pixel sigmas are modeled by the formula:
        
            sigma**2 = sigma0**2 + invgain * I
        
where I is the pixel value and "**2" means  the  square  of  the
        quantity.   If  either parameter is specified as INDEF or with a
        value less than zero then no sigma estimates are made and so  no
        error estimates for the measured parameters are made.
        
**NOTES** Not relevant for specviz because uncertainty handling is part of the fitting/etc process not a global property of the viz tool

### ? -- Page help information

*Description*:

**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used

### / -- Cycle through short status line help

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### :/help -- get help on GTOOLS options

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### :.help -- get help on standard cursor mode options

*Description*:


**NOTES** IRAF-style help is not relevant for jdaviz - the documentation, docstrings, or tooltips, etc should be used


### Calibration: star_name

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: mag

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: magband

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: teff

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: caldir

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Calibration: fnuzero

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### y -- Overplot standard star values from a calibration file.

*Description*:

**NOTES** Calibration functionality is out-of-scope for specviz, as it should be in pipelines or part of custom (typically notebook) workflows.

### Query: wavelength

*Description*:

**NOTES** Not relevant because anything that it is used for in IRAF would get a custom plugin input or a notebook keyword parameter.

### Query: linelist

*Description*:


**NOTES** Not relevant because anything that it is used for in IRAF would get a custom plugin input or a notebook keyword parameter.

### Query: wstart, wend, dw

*Description*:


**NOTES** Not relevant because anything that it is used for in IRAF would get a custom plugin input or a notebook keyword parameter.

### Query: boxsize (for smoothing)

*Description*:

**NOTES** Not relevant because anything that it is used for in IRAF would get a custom plugin input or a notebook keyword parameter.

# Temporary space 

In [ ]:
ss = app.data_collection.subset_groups[0].subsets[0].subset_state
ss.lo, ss.hi

### Code to import notebook content from Trello

In [ ]:
import requests

j = requests.get('https://trello.com/b/glMEUlE3.json').json()

In [ ]:
import nbformat as nbf

nb = nbf.read('Specviz from splot concept notebook.ipynb', nbf.NO_CONVERT)
nbfapi = getattr(nbf, 'v'+str(nb['nbformat']))

In [ ]:
for card in j['cards']:
    if card['name'] == 'IRAF Splot help text':
        continue
    markdown_text = '### '+ card['name'] + '\n\n*Description*:\n\n' + card['desc']
    nb['cells'].append(nbfapi.new_markdown_cell(markdown_text))
nbf.write(nb, 'output.ipynb')
# now manually move that to overwrite this file

In [ ]:
splot_help_file = requests.get(j['cards'][0]['attachments'][0]['url'])
print(splot_help_file.text)